In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

np.set_printoptions(linewidth=120, precision=5, suppress=True)

### Data

In [2]:
import pickle
from data_synthtext import GTUtility

#file_name = 'gt_util_synthtext_seglink.pkl'
file_name = 'gt_util_synthtext_seglink_xxs.pkl'
with open(file_name, 'rb') as f:
    gt_util = pickle.load(f)
gt_util_train, gt_util_val = gt_util.split(0.9)

In [3]:
batch_size = 16
image_size = (512, 512)

idxs, inputs, images, data = gt_util_val.sample_random_batch(batch_size=batch_size, input_size=image_size)

images = np.array(images, dtype='float32')

images_orig = [cv2.imread(os.path.join(gt_util_val.image_path, gt_util_val.image_names[idx])) for idx in idxs]

### Detection model

In [ ]:
from tbpp_model import TBPP512, TBPP512_dense
from tbpp_utils import PriorUtil

det_model = TBPP512_dense()
prior_util = PriorUtil(det_model)

weights_path = './checkpoints/201807091503_dsodtbpp512fl_synthtext/weights.018.h5'
confidence_threshold = 0.25

det_model.load_weights(weights_path)

### Recognition model

In [ ]:
from crnn_model import CRNN
from crnn_utils import alphabet87 as alphabet

input_width = 1024*2
input_height = 32

weights_path = './checkpoints/202002030820_crnn_cnn_synthtext_concat_continued/weights.200000.h5'

rec_model = CRNN((input_width, input_height, 1), len(alphabet), gru=False, cnn=True, prediction_only=True)
rec_model.load_weights(weights_path)

### Detection + Recognition model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Lambda

from tbpp_layers import TBPPDecodeAndCrop

x_in = Input(shape=det_model.input_shape[1:])
x = Lambda(lambda x: x*255)(x_in)
x_in_det = Lambda(lambda x: x-[104,117,123])(x)
x_det = det_model(x_in_det)
x_in_rec, x_det_dec = TBPPDecodeAndCrop(prior_util, 
                                        output_size=(input_height, input_width),
                                        confidence_threshold=0.7,
                                        iou_threshold=0.45, 
                                        top_k=200)([x, x_det])
x_rec = rec_model(x_in_rec)

model = Model([x_in,], [x_det_dec, x_in_rec, x_rec])

### Prediction

In [ ]:
from crnn_utils import decode

res = model.predict([images])
print([r.shape for r in res])

for i in range(batch_size):
    im = res[1][i]
    im = np.transpose(im, (1,0,2))
    plt.figure(figsize=(16,2))
    plt.imshow(im[:,:1000,0]/255, vmin=0, vmax=1, cmap='gray')
    plt.show()
    
    chars = [alphabet[c] for c in np.argmax(res[2][i], axis=1)]
    print(''.join(chars))
    
    offsets = [0] + [int(e//4) for e in res[0][i][:,19] if e > 0]
    print(offsets)
    res_strs = [ decode(chars[offsets[i]:offsets[i+1]]) for i in range(len(offsets)-1) ]
    res_strs = [ s.strip() for s in res_strs ]
    print(res_strs)

In [ ]:
%%timeit
res = model.predict([images[:1]])